# First import the ros python libraries

In [1]:
import rospy as rp

In [2]:
import rosnode
rosnode.get_node_names()

['/rosapi',
 '/rosout',
 '/pocketbot_81ea',
 '/turtlesim',
 '/adafruit_motor_hat_node',
 '/base2laser',
 '/pocketbot_9542',
 '/play_1577601841121188761',
 '/play_1577601574036218983',
 '/slamware_ros_sdk_server_node',
 '/velocity_muxer',
 '/rqt_gui_py_node_1',
 '/android_15/visualization_view',
 '/rosbridge_websocket',
 '/map2odom']

# Create a node in order to send and recieve ROS messages

In [3]:
rp.init_node('jupyter_demo')
rosnode.get_node_names()

['/rosapi',
 '/rosout',
 '/pocketbot_81ea',
 '/turtlesim',
 '/adafruit_motor_hat_node',
 '/base2laser',
 '/pocketbot_9542',
 '/play_1577601841121188761',
 '/play_1577601574036218983',
 '/slamware_ros_sdk_server_node',
 '/velocity_muxer',
 '/rqt_gui_py_node_1',
 '/android_15/visualization_view',
 '/jupyter_demo',
 '/rosbridge_websocket',
 '/map2odom']

In [4]:
topicInfo = rosnode.get_node_info_description('/turtlesim')
print(topicInfo)

Node [/turtlesim]
Publications: 
 * /rosout [rosgraph_msgs/Log]
 * /turtle1/color_sensor [turtlesim/Color]
 * /turtle1/pose [turtlesim/Pose]

Subscriptions: 
 * /turtle1/cmd_vel [unknown type]

Services: 
 * /clear
 * /kill
 * /reset
 * /spawn
 * /turtle1/set_pen
 * /turtle1/teleport_absolute
 * /turtle1/teleport_relative
 * /turtlesim/get_loggers
 * /turtlesim/set_logger_level



# List available topics

In [5]:
rp.get_published_topics()

[['/horizontal_laser_2d', 'sensor_msgs/MultiEchoLaserScan'],
 ['/pocketbot/waypoint', 'sensor_msgs/NavSatFix'],
 ['/slamware_ros_sdk_server_node/map_metadata', 'nav_msgs/MapMetaData'],
 ['/slamware_ros_sdk_server_node/scan', 'sensor_msgs/LaserScan'],
 ['/slamware_ros_sdk_server_node/basic_sensors_info',
  'slamware_ros_sdk/BasicSensorInfoArray'],
 ['/rosout_agg', 'rosgraph_msgs/Log'],
 ['/vertical_laser_2d', 'sensor_msgs/MultiEchoLaserScan'],
 ['/slamware_ros_sdk_server_node/map', 'nav_msgs/OccupancyGrid'],
 ['/pocketbot/fix', 'sensor_msgs/NavSatFix'],
 ['/client_count', 'std_msgs/Int32'],
 ['/pocketbot/heading', 'std_msgs/Float32'],
 ['/pocketbot/ai_speech_out', 'std_msgs/String'],
 ['/slamware_ros_sdk_server_node/virtual_walls',
  'slamware_ros_sdk/Line2DFlt32Array'],
 ['/pocketbot/emotion', 'std_msgs/String'],
 ['/imu', 'sensor_msgs/Imu'],
 ['/pocketbot/face', 'geometry_msgs/Twist'],
 ['/slamware_ros_sdk_server_node/robot_device_info',
  'slamware_ros_sdk/RobotDeviceInfo'],
 ['/cmd_

# Move the turtle by sending messages to the '/turtle1/cmd_vel' topic

In [6]:
import rostopic as rt
topicInfo = rt.get_info_text('/turtle1/cmd_vel')
print(topicInfo)

Type: geometry_msgs/Twist

Publishers: None

Subscribers: 
 * /turtlesim (http://192.168.2.66:34045/)




# The '/turtle1/cmd_vel' topic is a Twist type

In [7]:
topicType = rt.get_topic_type('/turtle1/cmd_vel')
print(topicType)

('geometry_msgs/Twist', '/turtle1/cmd_vel', None)


In [8]:
import rosmsg
msgInfo = rosmsg.get_msg_text(topicType[0])
print(msgInfo)

geometry_msgs/Vector3 linear
  float64 x
  float64 y
  float64 z
geometry_msgs/Vector3 angular
  float64 x
  float64 y
  float64 z



[WARN] [1577768430.106304]: Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


# Twist message consists of 2 Vector3 messages. <br> Each containing 3 floats. X, Y and Z. <br> One for Linear motion and the second for Angular motion.  

# Linear values are meters per second.<br>Angular values are radians per second.

# Create a publisher to send messages to a topic

In [9]:
from geometry_msgs.msg import Twist
publisher = rp.Publisher('turtle1/cmd_vel', Twist, queue_size=0)

# To move the turtle forward send a Twist message with a positive linear x value to the 'turtle/cmd_vel' topic

In [10]:
twistMessage = Twist()
twistMessage.linear.x = 5.0

publisher.publish(twistMessage)

# Negative values will move the turtle backwards

In [11]:
twistMessage.linear.x = -5.0
publisher.publish(twistMessage)


# Rotate counter clockwise by sending Twist message with angular z set to a positive value

In [12]:
twistMessage.linear.x = 5.0
twistMessage.angular.z = 6.2
publisher.publish(twistMessage)

# Clockwise with negative values

In [13]:
twistMessage.angular.z = -3.0
publisher.publish(twistMessage)

In [14]:
publisher.publish(twistMessage)

# Use standard python code to control the robot

In [15]:
vel_msg = Twist()

for n in range(5):
    vel_msg.linear.x = 5
    vel_msg.angular.z = 0
    publisher.publish(vel_msg)
    rp.sleep(0.5)
    for i in range(15):
        vel_msg.linear.x = 4 + n * n
        vel_msg.angular.z = 100 / (i + 10)
        publisher.publish(vel_msg)
        rp.sleep(0.25)

# Jupyter ROS can be used to create widgets to send messages

In [16]:
import jupyros as jr
from geometry_msgs.msg import Twist
jr.publish('/turtle1/cmd_vel', Twist)

### Right click to move the output to a new view

# Echo topic

In [17]:
def callback(msg):
    print(msg)
jr.subscribe('/turtle1/cmd_vel', Twist, callback)

In [18]:
import rosservice as rs
rs.get_service_list()

['/adafruit_motor_hat_node/get_loggers',
 '/base2laser/get_loggers',
 '/rosapi/delete_param',
 '/turtle1/teleport_relative',
 '/rosapi/action_servers',
 '/turtlesim/set_logger_level',
 '/rosapi/set_logger_level',
 '/rosbridge_websocket/set_logger_level',
 '/rosapi/service_host',
 '/rosbridge_websocket/get_loggers',
 '/rosapi/get_param_names',
 '/base2laser/set_logger_level',
 '/velocity_muxer/get_loggers',
 '/velocity_muxer/set_logger_level',
 '/rosapi/service_response_details',
 '/map2odom/get_loggers',
 '/rosapi/service_node',
 '/rosapi/services_for_type',
 '/adafruit_motor_hat_node/set_logger_level',
 '/play_1577601841121188761/pause_playback',
 '/turtlesim/get_loggers',
 '/rqt_gui_py_node_1/tf2_frames',
 '/rosapi/set_param',
 '/rosout/get_loggers',
 '/rosapi/get_param',
 '/rosapi/topics_for_type',
 '/rosapi/service_request_details',
 '/slamware_ros_sdk_server_node/sync_get_stcm',
 '/slamware_ros_sdk_server_node/get_loggers',
 '/rosapi/service_type',
 '/play_1577601841121188761/set_

In [19]:
from std_srvs.srv import Empty
reset = rp.ServiceProxy('/reset', Empty)
reset()

In [20]:
from IPython.core.display import display, HTML
import ipywidgets as iw
iw.HTML('<img src="https://placekitten.com/g/300/150">')

HTML(value='<img src="https://placekitten.com/g/300/150">')

In [21]:
from ipywidgets import IntSlider
slider = IntSlider(value=50)
slider

IntSlider(value=50)

In [22]:
javascript = '''
<script src="https://cdnjs.cloudflare.com/ajax/libs/nipplejs/0.7.3/nipplejs.min.js"></script>
<script>
    var options = {
        zone: document.getElementById('zone_joystick'),
        color: 'blue',
        mode: 'static',
        position: {top: '50%', left: '50%'}
    };
    var manager = nipplejs.create(options);
    manager.on('move', function(evt, data) {
        debug.innerHTML = JSON.stringify(data.position);
      })
</script>
<div id="zone_joystick" style="height:250px;">
<div id="debug"/>
</div>
'''

display(HTML(javascript))

In [23]:
from jupyros import ros3d
v = ros3d.Viewer()
rc = ros3d.ROSConnection()
tf_client = ros3d.TFClient(ros=rc, fixed_frame='/slamware_map')
v

Viewer()

In [24]:
laser_view = ros3d.LaserScan(topic="/slamware_ros_sdk_server_node/scan", ros=rc, tf_client=tf_client)
g = ros3d.GridModel()
v.objects = [g, laser_view]

In [27]:
laser_view.point_size = 1
laser_view.static_color = "green"

In [28]:
videoFeed = '<img src="http://192.168.2.84:8090/stream/video.mjpeg" width="60%"" height="60%"/>'
display(HTML(videoFeed))